In [ ]:
!apt-get install postgresql postgresql-contrib

In [ ]:
!/etc/init.d/postgresql start
import psycopg2
!sudo -u postgres createuser --superuser mak
!sudo -u postgres createdb ecom
!sudo -u postgres psql -c "alter user mak with encrypted password 'password';"

In [ ]:
conn = psycopg2.connect(dbname='ecom', user='mak', password='password', host='localhost')
cur = conn.cursor()
cur.execute('SELECT version()')
print(cur.fetchone())

In [ ]:
cur.execute('end;') 
cur.close() 
conn.close() 

In [ ]:
file = open("sample_data/tables","r")
l = []
table = ''''''
while True:
    line = file.readline()
    if not line:
        break
    if not line.startswith(");"):
      table += line
    else:
      l.append(table+");")
      table=""

file.close()

for i in l:
  cur.execute(i)

file = open("sample_data/data","r")
d = []
while True:
  line = file.readline()
  if not line:
    break
  d.append(line)

for i in d:
  cur.execute(i)


In [ ]:
drop = '''
DROP SCHEMA public CASCADE;
CREATE SCHEMA public;
GRANT ALL ON SCHEMA public TO postgres;
GRANT ALL ON SCHEMA public TO public;
COMMENT ON SCHEMA public IS 'standard public schema';
'''
cur.execute(drop)

In [ ]:
cur.execute("drop table customer cascade; drop table product cascade; drop table orders cascade; drop table seller cascade;")
cur.execute("drop table contains cascade; drop table sells cascade; drop table payment cascade; drop table address cascade;")

In [ ]:
t = "address"
info = f"select * from {t}"
cur.execute(info)
r = cur.fetchall()

for i in r:
  print(i)

In [ ]:
def attribute(t, entity):
  r = entity
  if t == "customer":
    for i in range(len(r)):
      r[i] = (str(r[i][0]), r[i][1].strip(), r[i][2].strip(), r[i][3].strip(),r[i][4].strip())
  elif t == "orders":
    for i in range(len(r)):
      r[i] = (str(r[i][0]), r[i][1].strip(), r[i][2].strip(), str(r[i][3]), str(r[i][4])) 
  elif t == "payment":
    for i in range(len(r)):
      r[i] = (str(r[i][0]), r[i][1].strip(), str(r[i][2]), str(r[i][3]))
  elif t == "product":
    for i in range(len(r)):
      r[i] = (str(r[i][0]), r[i][1].strip(), str(r[i][2]), str(r[i][3]))
  elif t == "contains":
    for i in range(len(r)):
      r[i] = (str(r[i][0]), str(r[i][1]), str(r[i][2]))
  elif t == "seller":
    for i in range(len(r)):
      r[i] = (str(r[i][0]), r[i][1].strip(), r[i][2].strip())
  elif t == "sells":
    for i in range(len(r)):
      r[i] = (str(r[i][0]), str(r[i][1]), str(r[i][2]))
  elif t == "address":
    for i in range(len(r)):
      r[i] = (r[i][0].strip(), r[i][1].strip(), r[i][2].strip(), r[i][3].strip(), str(r[i][4]), str(r[i][5]))

  return r

In [ ]:
def display(headers, r):
  # calculate the maximum length of each column
  max_length = [len(str(max(row, key=len))) for row in zip(headers, *r)]

  # print the headers
  print(' | '.join('{:<{}}'.format(header, max_length[i]) for i, header in enumerate(headers)))

  # print the rows
  for row in r:
      print(' | '.join('{:<{}}'.format(str(item), max_length[i]) for i, item in enumerate(row)))

In [ ]:
def show(table):
  x = f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table}';"
  cur.execute(x)
  h = cur.fetchall()
  t = tuple(elem for tup in h for elem in tup)
  info = f"select * from {table}"
  cur.execute(info)
  r = cur.fetchall()

  # display(t, attribute(table, r))
  
  for i in range(len(r)):
    for j in range(len(t)):
      if (isinstance(r[i][j], int)):
        temp = list(r[i])
        temp[j] = str(r[i][j])
        r[i]  = tuple(temp)
      else:
        temp = list(r[i])
        temp[j] = r[i][j].strip()
        r[i]  = tuple(temp)

  display(t, r)

In [ ]:
table="customer"
show(table)

In [ ]:
fun = '''
create or replace function get_stock(pid int)
returns int
language plpgsql
as 
$$
declare
p int;
begin
select sum(sells.stock) into p from sells inner join product on sells.product_id=product.product_id where product.product_id = pid;
return p;
end;
$$;
'''
cur.execute(fun)



In [ ]:
for i in range(501,526):
  funExe= f"select get_stock({i})"
  cur.execute(funExe)
  r = cur.fetchall()
  if r[0][0] != None:
    up = f'update product set stock = {r[0][0]} where product_id={i};'
    cur.execute(up)

In [ ]:
pro = '''
create or replace procedure get_amount(oid int)
language plpgsql
as
$$
declare 
cost int;
begin
cost = 0;
select sum(t.counts*p.price) into cost from 
(select c.counts, c.product_id 
from contains as c 
natural join orders as o
where o.order_id = oid) as t 
join product as p
on p.product_id = t.product_id;
if cost is not null then
  update orders
  set amount = cost where order_id = oid;
else
  update orders
  set amount = 0 where order_id = oid;
end if;
end;
$$;
'''

cur.execute(pro)

In [ ]:
for i in range(201, 226):
  proExe= f"call get_amount({i})"
  cur.execute(proExe)

# Data

# Insert Data

In [ ]:
insert into customer (customer_id, name, email, contact) values (101, 'Phylys', 'plillico0@auda.org.au', '8901220690');
insert into customer (customer_id, name, email, contact) values (102, 'Donn', 'dcorzon1@sina.com.cn', '8917271841');
insert into customer (customer_id, name, email, contact) values (103, 'Laurens', 'lscandroot2@hexun.com', '1914665357');
insert into customer (customer_id, name, email, contact) values (104, 'Margaretha', 'mreddel3@bbb.org', '2853297106');
insert into customer (customer_id, name, email, contact) values (105, 'Gris', 'gchaize4@pcworld.com', '2619139640');
insert into customer (customer_id, name, email, contact) values (106, 'Delphinia', 'dephcon@att.org', '9624696215');
insert into customer (customer_id, name, email, contact) values (107, 'Lenore', 'lpavie6@livejournal.com', '7235905306');
insert into customer (customer_id, name, email, contact) values (108, 'Brodie', 'bpantin7@msu.edu', '9589004089');
insert into customer (customer_id, name, email, contact) values (109, 'Eben', 'esooper8@harvard.edu', '7457959323');
insert into customer (customer_id, name, email, contact) values (110, 'Claudetta', 'ccausier9@nih.gov', '5091881242');
insert into customer (customer_id, name, email, contact) values (111, 'Helyn', 'hkinneira@google.com.br', '5565306558');
insert into customer (customer_id, name, email, contact) values (112, 'Ursula', 'udelbouxb@freewebs.com', '8282222648');
insert into customer (customer_id, name, email, contact) values (113, 'Lucy', 'lhaddonc@columbia.edu', '4159231486');
insert into customer (customer_id, name, email, contact) values (114, 'Kai', 'ktippertond@flickr.com', '5551807258');
insert into customer (customer_id, name, email, contact) values (115, 'Hamlin', 'hkivelle@list-manage.com', '6073366947');
insert into customer (customer_id, name, email, contact) values (116, 'Rubi', 'rpaullinf@techcrunch.com', '5429141664');
insert into customer (customer_id, name, email, contact) values (117, 'Gerome', 'gcrisallg@state.tx.us', '3904698038');
insert into customer (customer_id, name, email, contact) values (118, 'Ashly', 'athomkeh@github.io', '2952974244');
insert into customer (customer_id, name, email, contact) values (119, 'Otha', 'odarvillei@live.com', '9197033421');
insert into customer (customer_id, name, email, contact) values (120, 'Zelig', 'zhullbrookj@bizjournals.com', '4698551918');
insert into customer (customer_id, name, email, contact) values (121, 'Darrell', 'dshuxsmithk@columbia.edu', '7472692252');
insert into customer (customer_id, name, email, contact) values (122, 'Madison', 'mpadulal@phpbb.com', '7367037379');
insert into customer (customer_id, name, email, contact) values (123, 'Holly', 'whiteholy@walt.us', '7373031400');
insert into customer (customer_id, name, email, contact) values (124, 'Caterina', 'cbrodien@furl.net', '4351471128');
insert into customer (customer_id, name, email, contact) values (125, 'Bogey', 'bcastroo@aol.com', '5547639198');
insert into product (product_id, name, price) values (201, 'umbrella', 26.2);
insert into product (product_id, name, price) values (202, 'charger', 32.9);
insert into product (product_id, name, price) values (203, 'sneakers', 28.1);
insert into product (product_id, name, price) values (204, 'popcorn', 24.59);
insert into product (product_id, name, price) values (205, 'sneakers', 22.6);
insert into product (product_id, name, price) values (206, 'microphone', 38.73);
insert into product (product_id, name, price) values (207, 'perfume', 26.57);
insert into product (product_id, name, price) values (208, 'slippers', 32.44);
insert into product (product_id, name, price) values (209, 'wallet', 11.54);
insert into product (product_id, name, price) values (210, 'lamp', 33.01);
insert into product (product_id, name, price) values (211, 'microphone', 30.98);
insert into product (product_id, name, price) values (212, 'candle', 47.86);
insert into product (product_id, name, price) values (213, 'microphone', 49.46);
insert into product (product_id, name, price) values (214, 'microphone', 25.27);
insert into product (product_id, name, price) values (215, 'conditioner', 34.52);
insert into product (product_id, name, price) values (216, 'sailboat', 11.18);
insert into product (product_id, name, price) values (217, 'dumbbell', 46.51);
insert into product (product_id, name, price) values (218, 'keyboard', 49.58);
insert into product (product_id, name, price) values (219, 'beardtrimmer', 45.44);
insert into product (product_id, name, price) values (220, 'sweater', 25.43);
insert into product (product_id, name, price) values (221, 'conditioner', 39.43);
insert into product (product_id, name, price) values (222, 'pillow', 42.87);
insert into product (product_id, name, price) values (223, 'lipstick', 30.81);
insert into product (product_id, name, price) values (224, 'popcorn', 39.34);
insert into product (product_id, name, price) values (225, 'lipstick', 10.49);
insert into orders (order_id, status, date, customer_id) values (301, 'Delivered', '23-11-2022', 110);
insert into orders (order_id, status, date, customer_id) values (302, 'Ordered', '18-11-2022', 104);
insert into orders (order_id, status, date, customer_id) values (303, 'Cancelled', '30-12-2022', 114);
insert into orders (order_id, status, date, customer_id) values (304, 'Delivered', '29-12-2022', 110);
insert into orders (order_id, status, date, customer_id) values (305, 'Ordered', '10-05-2022', 107);
insert into orders (order_id, status, date, customer_id) values (306, 'Delivered', '02-04-2022', 116);
insert into orders (order_id, status, date, customer_id) values (307, 'Cancelled', '20-11-2022', 107);
insert into orders (order_id, status, date, customer_id) values (308, 'Ordered', '30-11-2022', 104);
insert into orders (order_id, status, date, customer_id) values (309, 'Delivered', '27-10-2022', 122);
insert into orders (order_id, status, date, customer_id) values (310, 'Cancelled', '23-11-2022', 102);
insert into address (apartment, street, city, state, pincode, customer_id) values (82, 'Stephen', 'Gaithersburg', 'Maryland', '20883', 109);
insert into address (apartment, street, city, state, pincode, customer_id) values (30, 'Barnett', 'Omaha', 'Nebraska', '68179', 119);
insert into address (apartment, street, city, state, pincode, customer_id) values (66, 'Porter', 'Austin', 'Texas', '78715', 105);
insert into address (apartment, street, city, state, pincode, customer_id) values (93, 'Holy Cross', 'Austin', 'Texas', '78778', 102);
insert into address (apartment, street, city, state, pincode, customer_id) values (93, 'Gulseth', 'Worcester', 'Massachusetts', '01610', 115);
insert into address (apartment, street, city, state, pincode, customer_id) values (21, 'Dryden', 'Plano', 'Texas', '75074', 115);
insert into address (apartment, street, city, state, pincode, customer_id) values (99, 'Park Meadow', 'Lexington', 'Kentucky', '40576', 125);
insert into address (apartment, street, city, state, pincode, customer_id) values (43, 'Kings', 'Amarillo', 'Texas', '79171', 122);
insert into address (apartment, street, city, state, pincode, customer_id) values (98, 'Mallard', 'Topeka', 'Kansas', '66622', 113);
insert into address (apartment, street, city, state, pincode, customer_id) values (31, 'Hermina', 'Detroit', 'Michigan', '48275', 115);
insert into address (apartment, street, city, state, pincode, customer_id) values (18, 'Rigney', 'Colorado Springs', 'Colorado', '80915', 117);
insert into address (apartment, street, city, state, pincode, customer_id) values (10, '3rd', 'Houston', 'Texas', '77206', 121);
insert into address (apartment, street, city, state, pincode, customer_id) values (14, 'Hooker', 'Akron', 'Ohio', '44329', 105);
insert into address (apartment, street, city, state, pincode, customer_id) values (92, 'Garrison', 'Amarillo', 'Texas', '79116', 107);
insert into address (apartment, street, city, state, pincode, customer_id) values (60, 'Springs', 'Springfield', 'Illinois', '62776', 125);
insert into address (apartment, street, city, state, pincode, customer_id) values (64, 'Fisk', 'Fairbanks', 'Alaska', '99709', 107);
insert into address (apartment, street, city, state, pincode, customer_id) values (10, 'Lakeland', 'Kansas City', 'Missouri', '64149', 102);
insert into address (apartment, street, city, state, pincode, customer_id) values (22, 'Lake View', 'Pocatello', 'Idaho', '83206', 119);
insert into address (apartment, street, city, state, pincode, customer_id) values (79, 'Blue Bill Park', 'San Diego', 'California', '92132', 101);
insert into address (apartment, street, city, state, pincode, customer_id) values (94, 'Cardinal', 'Shawnee Mission', 'Kansas', '66286', 120);
insert into address (apartment, street, city, state, pincode, customer_id) values (57, 'Sundown', 'Van Nuys', 'California', '91411', 110);
insert into address (apartment, street, city, state, pincode, customer_id) values (40, 'Maryland', 'Decatur', 'Georgia', '30089', 102);
insert into address (apartment, street, city, state, pincode, customer_id) values (19, 'Raven', 'Omaha', 'Nebraska', '68124', 115);
insert into address (apartment, street, city, state, pincode, customer_id) values (78, 'Atwood', 'Houston', 'Texas', '77095', 105);
insert into address (apartment, street, city, state, pincode, customer_id) values (45, 'Roth', 'Jackson', 'Mississippi', '39296', 123);
insert into payment (payment_id, mode, order_id) values (401, 'Wallet', 301);
insert into payment (payment_id, mode, order_id) values (402, 'Card', 302);
insert into payment (payment_id, mode, order_id) values (403, 'Card', 303);
insert into payment (payment_id, mode, order_id) values (404, 'UPI', 304);
insert into payment (payment_id, mode, order_id) values (405, 'Wallet', 305);
insert into payment (payment_id, mode, order_id) values (406, 'UPI', 306);
insert into payment (payment_id, mode, order_id) values (407, 'UPI', 307);
insert into payment (payment_id, mode, order_id) values (408, 'COD', 308);
insert into payment (payment_id, mode, order_id) values (409, 'COD', 309);
insert into payment (payment_id, mode, order_id) values (410, 'COD', 310);
insert into seller (seller_id, name, contact) values (501, 'Padraic', '4869419743');
insert into seller (seller_id, name, contact) values (502, 'Fonsie', '1313871017');
insert into seller (seller_id, name, contact) values (503, 'Joanne', '3557747088');
insert into seller (seller_id, name, contact) values (504, 'Connie', '2862058724');
insert into seller (seller_id, name, contact) values (505, 'Garrott', '2392229497');
insert into seller (seller_id, name, contact) values (506, 'Annaliese', '1538408041');
insert into seller (seller_id, name, contact) values (507, 'Thaddus', '5587487280');
insert into seller (seller_id, name, contact) values (508, 'Stormi', '8966025352');
insert into seller (seller_id, name, contact) values (509, 'Tailor', '4831349930');
insert into seller (seller_id, name, contact) values (510, 'Marita', '8953396485');
insert into seller (seller_id, name, contact) values (511, 'Meggi', '7381083012');
insert into seller (seller_id, name, contact) values (512, 'Rory', '9448841441');
insert into seller (seller_id, name, contact) values (513, 'Cristian', '4656114049');
insert into seller (seller_id, name, contact) values (514, 'Micaela', '9947660260');
insert into seller (seller_id, name, contact) values (515, 'Allix', '1302244777');
insert into seller (seller_id, name, contact) values (516, 'Brod', '2026826916');
insert into seller (seller_id, name, contact) values (517, 'Fionna', '8823138943');
insert into seller (seller_id, name, contact) values (518, 'Tedra', '7255344059');
insert into seller (seller_id, name, contact) values (519, 'Art', '5923215601');
insert into seller (seller_id, name, contact) values (520, 'Daffy', '8037305246');
insert into seller (seller_id, name, contact) values (521, 'Katie', '7841105483');
insert into seller (seller_id, name, contact) values (522, 'Patrizia', '1156916231');
insert into seller (seller_id, name, contact) values (523, 'Keenan', '7841117567');
insert into seller (seller_id, name, contact) values (524, 'Haze', '1448175995');
insert into seller (seller_id, name, contact) values (525, 'Lyda', '6869030449');
insert into sells (seller_id, product_id, stock) values (515, 224, 10);
insert into sells (seller_id, product_id, stock) values (501, 223, 2);
insert into sells (seller_id, product_id, stock) values (515, 223, 12);
insert into sells (seller_id, product_id, stock) values (511, 220, 13);
insert into sells (seller_id, product_id, stock) values (506, 207, 0);
insert into sells (seller_id, product_id, stock) values (507, 201, 0);
insert into sells (seller_id, product_id, stock) values (513, 217, 6);
insert into sells (seller_id, product_id, stock) values (519, 207, 15);
insert into sells (seller_id, product_id, stock) values (504, 208, 6);
insert into sells (seller_id, product_id, stock) values (505, 210, 2);
insert into sells (seller_id, product_id, stock) values (508, 216, 5);
insert into sells (seller_id, product_id, stock) values (523, 212, 11);
insert into sells (seller_id, product_id, stock) values (506, 215, 10);
insert into sells (seller_id, product_id, stock) values (508, 205, 6);
insert into sells (seller_id, product_id, stock) values (516, 210, 2);
insert into sells (seller_id, product_id, stock) values (518, 217, 11);
insert into sells (seller_id, product_id, stock) values (518, 216, 8);
insert into sells (seller_id, product_id, stock) values (502, 207, 13);
insert into sells (seller_id, product_id, stock) values (523, 209, 10);
insert into sells (seller_id, product_id, stock) values (522, 217, 10);
insert into sells (seller_id, product_id, stock) values (517, 206, 1);
insert into sells (seller_id, product_id, stock) values (514, 213, 4);
insert into sells (seller_id, product_id, stock) values (508, 224, 12);
insert into sells (seller_id, product_id, stock) values (522, 213, 5);
insert into sells (seller_id, product_id, stock) values (511, 223, 8);
insert into contains (order_id, counts, product_id) values (301, 14, 218);
insert into contains (order_id, counts, product_id) values (302, 12, 223);
insert into contains (order_id, counts, product_id) values (310, 9, 219);
insert into contains (order_id, counts, product_id) values (303, 6, 221);
insert into contains (order_id, counts, product_id) values (301, 13, 223);
insert into contains (order_id, counts, product_id) values (306, 14, 210);
insert into contains (order_id, counts, product_id) values (301, 11, 209);
insert into contains (order_id, counts, product_id) values (304, 1, 205);
insert into contains (order_id, counts, product_id) values (308, 2, 214);
insert into contains (order_id, counts, product_id) values (307, 6, 220);
insert into contains (order_id, counts, product_id) values (309, 5, 219);
insert into contains (order_id, counts, product_id) values (303, 14, 211);
insert into contains (order_id, counts, product_id) values (307, 11, 216);
insert into contains (order_id, counts, product_id) values (304, 7, 221);
insert into contains (order_id, counts, product_id) values (301, 1, 204);
insert into contains (order_id, counts, product_id) values (303, 14, 204);
insert into contains (order_id, counts, product_id) values (305, 15, 214);
insert into contains (order_id, counts, product_id) values (307, 8, 204);
insert into contains (order_id, counts, product_id) values (310, 4, 221);
insert into contains (order_id, counts, product_id) values (301, 6, 210);
insert into contains (order_id, counts, product_id) values (308, 7, 202);
insert into contains (order_id, counts, product_id) values (309, 2, 214);
insert into contains (order_id, counts, product_id) values (310, 13, 218);
insert into contains (order_id, counts, product_id) values (305, 4, 203);
insert into contains (order_id, counts, product_id) values (301, 1, 210);